In [1]:
import os

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import time
%matplotlib inline

In [2]:
Ten2PIL = torchvision.transforms.ToPILImage()
PIL2Ten = torchvision.transforms.ToTensor()

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [4]:
dataset = torchvision.datasets.CIFAR10(root='data', train=True, transform=transform_train, download=True)
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, lengths=[int(0.9 * len(dataset)), len(dataset) - int(0.9*len(dataset))])
test_dataset = torchvision.datasets.CIFAR10(root='data', train=False, transform=transform_test, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [6]:
device = torch.device('cuda')

In [7]:
class modelClass(nn.Module):
    def __init__(self, num_classes):
        super(modelClass, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.mp = nn.MaxPool2d(kernel_size=2)
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        # self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2)
        # self.fc0 = nn.Linear(512, 128)
        self.fc1 = nn.Linear(576, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        out = self.mp(F.relu(self.conv0(x)))
        out = self.mp(F.relu(self.conv1(out)))
        # out = self.mp(F.relu(self.conv2(out)))
        out = nn.Flatten()(out)
        # print(out.shape)
        # out = self.fc0(out)
        out = self.fc1(out)
        return self.fc2(out)

In [8]:
model = modelClass(num_classes=10).to(device) # torchvision.models.mobilenet_v2(num_classes=10).to(device) 

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
lossfunc = nn.CrossEntropyLoss()

In [11]:
num_epoch = 300
best_accu = 0
cnt = 0
model.train()
for epoch in range(num_epoch):
    for i, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = lossfunc(pred, y)
        loss.backward()
        optimizer.step()
        if i % 500 == 0 or i == (len(train_dataset) / 32) - 1 :
            # param = next(model.parameters())
            # print(param.grad.mean() / param.mean())
            model.eval()
            with torch.no_grad():
                valid_loss, accu = 0, 0
                for j, (x, y) in enumerate(valid_loader):
                    x, y = x.to(device), y.to(device)
                    pred = model(x)
                    valid_loss += lossfunc(pred, y)
                    accu += (pred.argmax(-1) == y.squeeze()).sum().item()
                # print(accu, valid_loss)
                valid_loss /= j
                accu /= len(valid_dataset)
                del x, y, pred
            print("Epoch: {} | Train loss: {:.4f} | Valid loss: {:.4f} | Valid Accu: {:.4f}".format(
                epoch, loss.item(), valid_loss.item(), accu
            ))
            model.train()
    # print("\n {:.4f} {:.4f} {} \n".format(1.02 * best_accu, accu, cnt))
    if accu >= best_accu:
        cnt = 0
        best_accu = accu
        torch.save(model.state_dict(), os.path.join('model', 'cifar',
                                                    'model-{}.pt'.format(epoch)))
    else:
        cnt += 1

    if cnt == 50:
        # print("Early stopped!")
        break

Epoch: 0 | Train loss: 2.2794 | Valid loss: 2.3200 | Valid Accu: 0.1206
Epoch: 0 | Train loss: 2.2486 | Valid loss: 2.2572 | Valid Accu: 0.1647
Epoch: 0 | Train loss: 2.1924 | Valid loss: 2.1920 | Valid Accu: 0.1965
Epoch: 1 | Train loss: 2.2870 | Valid loss: 2.1410 | Valid Accu: 0.2362
Epoch: 1 | Train loss: 2.0640 | Valid loss: 2.0833 | Valid Accu: 0.2596
Epoch: 1 | Train loss: 2.0645 | Valid loss: 2.0198 | Valid Accu: 0.2908
Epoch: 2 | Train loss: 1.8485 | Valid loss: 1.9858 | Valid Accu: 0.2962
Epoch: 2 | Train loss: 2.1216 | Valid loss: 1.9530 | Valid Accu: 0.3059
Epoch: 2 | Train loss: 1.9905 | Valid loss: 1.9240 | Valid Accu: 0.3146
Epoch: 3 | Train loss: 2.1176 | Valid loss: 1.9012 | Valid Accu: 0.3263
Epoch: 3 | Train loss: 1.8707 | Valid loss: 1.8742 | Valid Accu: 0.3339
Epoch: 3 | Train loss: 2.0603 | Valid loss: 1.8426 | Valid Accu: 0.3465
Epoch: 4 | Train loss: 1.9821 | Valid loss: 1.8217 | Valid Accu: 0.3543


KeyboardInterrupt: 

In [23]:
best_accu

0.8125

In [24]:
loaded_model = torchvision.models.mobilenet_v2(num_classes=10).to(device)

In [25]:
loaded_model.load_state_dict(torch.load(os.path.join('model', 'cifar', 'model-20.pt')))
loaded_model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [26]:
test_accu = 0
for _, (x, y) in enumerate(test_loader):
    x, y = x.to(device), y.to(device)
    pred = loaded_model(x)
    test_accu += (pred.argmax(-1) == y).sum().item()

In [28]:
test_accu / len(test_dataset)

0.5316